In [1]:
import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms
from tqdm import tqdm

In [2]:
from TransUNet.utils import DiceLoss

In [3]:
import cv2, ast
import torchvision
import matplotlib.pyplot as plt
import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
from torchvision import transforms
from torch.autograd import Variable
import torch.functional as F
import pickle
import torch.nn.functional as F
from PIL import Image, ImageFilter
import pydicom
from skimage import measure
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw
import copy
from scipy.ndimage import rotate
import random
import seaborn as sns
from skimage import exposure

In [4]:
val_df = pd.read_pickle('lung_cancer_test.pkl')
val_df.head()

,label1,mask,hu_array,hu_array_old
0,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
1,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
2,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
3,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
4,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."


In [5]:
from TransUNet.networks.vit_seg_modeling import VisionTransformer as ViT_seg
from TransUNet.networks.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg
config_vit = CONFIGS_ViT_seg['R50-ViT-L_16']
config_vit.n_classes = 2
config_vit.n_skip = 3
img_size = 512
vit_patches_size = 16
config_vit.patches.grid = (int(img_size / vit_patches_size), int(img_size / vit_patches_size))
net = ViT_seg(config_vit, img_size=img_size, num_classes=config_vit.n_classes).cuda()
#net.load_from(weights=np.load(config_vit.pretrained_path))

In [6]:
net.load_state_dict(torch.load('U-Net with VGG/trans-u-net-R50-base-512_withdataaugn.pth'))

<All keys matched successfully>

In [7]:
net.eval()

VisionTransformer(
  (transformer): Transformer(
    (embeddings): Embeddings(
      (hybrid_model): ResNetV2(
        (root): Sequential(
          (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (gn): GroupNorm(32, 64, eps=1e-06, affine=True)
          (relu): ReLU(inplace=True)
        )
        (body): Sequential(
          (block1): Sequential(
            (unit1): PreActBottleneck(
              (gn1): GroupNorm(32, 64, eps=1e-06, affine=True)
              (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (gn2): GroupNorm(32, 64, eps=1e-06, affine=True)
              (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (gn3): GroupNorm(32, 256, eps=1e-06, affine=True)
              (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (relu): ReLU(inplace=True)
              (downsample): StdConv2d(64, 256, 

In [8]:
def calculate_dice_coefficient(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred)    
    dice_coefficient = (2.0 * intersection) / (union + 1e-8)     
    return dice_coefficient
def calculate_iou(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred) - intersection   
    iou = (intersection + 1e-8) / (union + 1e-8)     
    return iou
def calculate_iou(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred) - intersection   
    iou = (intersection + 1e-8) / (union + 1e-8)      
    return iou
def precision_score(groundtruth_mask, pred_mask):
    true_positives = np.sum(pred_mask * groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = (true_positives + 1e-8) / (total_pixel_pred + 1e-8)
    return precision
def recall_score(groundtruth_mask, pred_mask):
    true_positives = np.sum(pred_mask * groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = (true_positives + 1e-8) / (total_pixel_truth + 1e-8)
    return recall

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
d = []
iou = []
prec = []
rec = []
labels = val_df['label1'].unique()
for j in labels:
    dices = []
    test = val_df.loc[val_df['label1'] == j].reset_index(drop=True)
    for i in range(len(test)):
        img = test['hu_array'][i]
        img = (img-np.min(img))/(np.max(img)-np.min(img))
        img = exposure.equalize_adapthist(img/np.max(img))
        img = transforms.ToTensor()(img)
        img = img.float().cuda()  
        out = net(img.unsqueeze(0))  
        out = torch.softmax(out, dim=1)
        mask = test['mask'][i]   
        masks = (mask > 0.5).astype(int)
        try:
            mask_pred = (out[0][1].cpu().detach().numpy() > 0.5)**2
        except:
            mask_pred = np.zeros([512,512])
        dices.append(calculate_dice_coefficient(masks, mask_pred))
        d.append(calculate_dice_coefficient(masks, mask_pred))
        iou.append(calculate_iou(masks, mask_pred))
        prec.append(precision_score(masks, mask_pred))
        rec.append(recall_score(masks, mask_pred))
    print("Dice for " + j + " " + str(np.mean(dices))+" " +str(len(dices)))
print("Average") 
print("DICE " + str(np.mean(d)))
print("IoU " + str(np.mean(iou)))
print("precision " + str(np.mean(prec)))
print("recall "+str(np.mean(rec)))

Dice for LR2 0.10003961098749836 35
Dice for LR3 0.37832149072381266 37
Dice for LR4A 0.45291734407963113 97
Dice for LR4B 0.5889898933686388 95
Average
DICE 0.4446450901026077
IoU 0.39008079215403046
precision 0.8546991933575012
recall 0.43647317457704454


In [10]:
sum(p.numel() for p in net.parameters() if p.requires_grad)

324871586